In [1]:
import graphene
import pyodbc
from graphene import ObjectType, String, Schema, List, Field,InputObjectType
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
server = 'USKOLNIKRANJAN1'
database = 'AdventureWorks2014'
username = 'US\\nikranjan' 
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection=yes')

In [3]:
def execute_query(query):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        return result
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        return []

In [4]:
class User(ObjectType):
    facultyID = String()
    name = String()
    department = String()
    country = String()
    city = String()
    address = String()
    appointment = String()

In [5]:
class Query(ObjectType):
    users = List(User)

    def resolve_users(self, info):
        query = "SELECT FACULTY_ID, Name, Department, COUNTRY, City, ADDRESS, APPOINTMENT FROM AdventureWorks2014.dbo.FACULTY"
        result = execute_query(query)
        return [User(
            facultyID=row[0],
            name=row[1],
            department=row[2],
            country=row[3],
            city=row[4],
            address=row[5],
            appointment=row[6]
        ) for row in result]

schema = Schema(query=Query)

In [6]:
query = '''
query {
    users {
        facultyID
        name
        department
        country
        city
        address
        appointment
    }
}
'''

result = schema.execute(query)
print(result.data)

{'users': [{'facultyID': 'F1001', 'name': 'John Write', 'department': 'EEE', 'country': 'US', 'city': 'New York', 'address': '123 Main Street', 'appointment': '2023-05-26'}, {'facultyID': 'F1002', 'name': 'Jane Smith', 'department': 'Mechanical', 'country': 'US', 'city': 'Los Angeles', 'address': '456 Oak Avenue', 'appointment': '2023-05-27'}, {'facultyID': 'F1003', 'name': 'Robert Johnson', 'department': 'EEE', 'country': 'US', 'city': 'Chicago', 'address': '789 Elm Street', 'appointment': '2023-05-28'}, {'facultyID': 'F1004', 'name': 'Wilson Doug', 'department': 'EEE', 'country': 'US', 'city': 'Chicago', 'address': 'EKLUTNA LAKE RD', 'appointment': '2023-05-29'}, {'facultyID': 'F1005', 'name': 'Mary Kom', 'department': 'CSE', 'country': 'US', 'city': 'Lynn', 'address': '70 Pleasant Valley Street', 'appointment': '2023-05-30'}, {'facultyID': 'F1006', 'name': 'Andrew Rode', 'department': 'Information Technology', 'country': 'US', 'city': 'North Adams', 'address': '830 Curran Memorial H

In [7]:
query = 'SELECT * FROM AdventureWorks2014.dbo.FACULTY'
df = pd.read_sql_query(query, conn)
df.head()

,FACULTY_ID,Name,Department,COUNTRY,City,ADDRESS,APPOINTMENT
0,F1001,John Write,EEE,US,New York,123 Main Street,2023-05-26
1,F1002,Jane Smith,Mechanical,US,Los Angeles,456 Oak Avenue,2023-05-27
2,F1003,Robert Johnson,EEE,US,Chicago,789 Elm Street,2023-05-28
3,F1004,Wilson Doug,EEE,US,Chicago,EKLUTNA LAKE RD,2023-05-29
4,F1005,Mary Kom,CSE,US,Lynn,70 Pleasant Valley Street,2023-05-30


In [8]:
class Query(ObjectType):
    users = List(User, facultyID=String()) 

    def resolve_users(self, info, facultyID=None): 
        query = "SELECT FACULTY_ID, Name, Department, COUNTRY, City, ADDRESS, APPOINTMENT FROM AdventureWorks2014.dbo.FACULTY"
        if facultyID:
            query += f" WHERE FACULTY_ID = '{facultyID}'"

        result = execute_query(query)
        return [User(
            facultyID=row[0],
            name=row[1],
            department=row[2],
            country=row[3],
            city=row[4],
            address=row[5],
            appointment=row[6]
        ) for row in result]

In [9]:
schema = Schema(query=Query)
query = '''
query {
    users(facultyID: "F1001") {
        facultyID
        name
        department
        country
        city
        address
        appointment
    }
}
'''
result = schema.execute(query)
print(result.data)

{'users': [{'facultyID': 'F1001', 'name': 'John Write', 'department': 'EEE', 'country': 'US', 'city': 'New York', 'address': '123 Main Street', 'appointment': '2023-05-26'}]}


In [10]:
import graphene
import pyodbc

from graphene import ObjectType, String, Schema, List, InputObjectType

server = 'USKOLNIKRANJAN1'
database = 'AdventureWorks2014'
username = 'US\\nikranjan' 
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection=yes')

def execute_query(query):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        return result
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        return []

class User(ObjectType):
    facultyID = String()
    name = String()
    department = String()
    country = String()
    city = String()
    address = String()
    appointment = String()
    email = String()
    age = String()


class Query(ObjectType):
    users = List(User, facultyID=String())

    def resolve_users(self, info, facultyID=None):
        query = "SELECT FACULTY_ID, Name, Department, COUNTRY, City, ADDRESS, APPOINTMENT FROM AdventureWorks2014.dbo.FACULTY"
        if facultyID:
            query += f" WHERE FACULTY_ID = '{facultyID}'"

        result = execute_query(query)
        users = []
        for row in result:
            user = User(
                facultyID=row[0],
                name=row[1],
                department=row[2],
                country=row[3],
                city=row[4],
                address=row[5],
                appointment=row[6],
            )
            users.append(user)

        return users

class FacultyInput(InputObjectType):
    email = graphene.String(required=True)
    age = graphene.String(required=True)

class CreateFaculty(graphene.Mutation):
    class Arguments:
        faculty_data = FacultyInput(required=True)

    user = graphene.Field(User)

   
    def mutate(root, info, faculty_data=None):

        user = User(
            email=faculty_data.email,
            age=faculty_data.age
        )
        return CreateFaculty(user=user)


class Mutation(graphene.ObjectType):
    create_faculty = CreateFaculty.Field()


schema = Schema(query=Query, mutation=Mutation)

query = '''
    mutation(email: $email, $age: String!) {
        createFaculty(facultyData: { email: $email, age: $age }) {
            user {
                email
                age
            }
        }
    }
'''

variables = {
    "email": "peter@xyz.com",
    "age": "30"
}

result = schema.execute(query, variables=variables)
print(result.data)


None


In [11]:
import graphene
import pyodbc
import hashlib
from flask import Flask, jsonify, request
from graphene import ObjectType, String, Schema, List, Field
import warnings
from starlette.applications import Starlette
from starlette_graphene3 import GraphQLApp, make_graphiql_handler
import uvicorn

warnings.filterwarnings("ignore")

server = 'USKOLNIKRANJAN1'
database = 'AdventureWorks2014'
username = 'US\\nikranjan' 
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection=yes')

persisted_queries = {}

def generate_query_id(query):
    return hashlib.md5(query.encode('utf-8')).hexdigest()

def execute_query(query):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        return result
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        return []

class User(ObjectType):
    facultyID = String()
    name = String()
    department = String()
    country = String()
    city = String()
    address = String()
    appointment = String()

class Query(ObjectType):
    users = List(User)

    def resolve_users(self, info):
        query = '''
            SELECT FACULTY_ID, Name, Department, COUNTRY, City, ADDRESS, APPOINTMENT
            FROM AdventureWorks2014.dbo.FACULTY
        '''
        query_id = generate_query_id(query)
        persisted_queries[query_id] = query

        result = execute_query(query)
        return [User(
            facultyID=row[0],
            name=row[1],
            department=row[2],
            country=row[3],
            city=row[4],
            address=row[5],
            appointment=row[6]
        ) for row in result]

schema = Schema(query=Query)
query = '''
query {
    users {
        facultyID
        name
        department
        country
        city
        address
        appointment
    }
}
'''

result = schema.execute(query)
print(result.data)

{'users': [{'facultyID': 'F1001', 'name': 'John Write', 'department': 'EEE', 'country': 'US', 'city': 'New York', 'address': '123 Main Street', 'appointment': '2023-05-26'}, {'facultyID': 'F1002', 'name': 'Jane Smith', 'department': 'Mechanical', 'country': 'US', 'city': 'Los Angeles', 'address': '456 Oak Avenue', 'appointment': '2023-05-27'}, {'facultyID': 'F1003', 'name': 'Robert Johnson', 'department': 'EEE', 'country': 'US', 'city': 'Chicago', 'address': '789 Elm Street', 'appointment': '2023-05-28'}, {'facultyID': 'F1004', 'name': 'Wilson Doug', 'department': 'EEE', 'country': 'US', 'city': 'Chicago', 'address': 'EKLUTNA LAKE RD', 'appointment': '2023-05-29'}, {'facultyID': 'F1005', 'name': 'Mary Kom', 'department': 'CSE', 'country': 'US', 'city': 'Lynn', 'address': '70 Pleasant Valley Street', 'appointment': '2023-05-30'}, {'facultyID': 'F1006', 'name': 'Andrew Rode', 'department': 'Information Technology', 'country': 'US', 'city': 'North Adams', 'address': '830 Curran Memorial H

In [12]:
query = '''
query {
  users {
    facultyID
    name
    department
    
  }
}
'''
query_id = generate_query_id(query)
print(query_id)

1ac42a494d90f002896d330a1ca7d0a7


In [13]:
queries = [{"id": query_id, "query": query} for query_id, query in persisted_queries.items()]
df_queries = pd.DataFrame(queries)
df_queries.to_json("persisted_document.json", orient="records")

In [2]:
import graphene
import pyodbc
import hashlib
from flask import Flask, jsonify, request
from graphene import ObjectType, String, Schema, List, Field
import warnings
from starlette.applications import Starlette
from starlette_graphene3 import GraphQLApp
import uvicorn
from graphql import parse

warnings.filterwarnings("ignore")

server = 'USKOLNIKRANJAN1'
database = 'AdventureWorks2014'
username = 'US\\nikranjan' 
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection=yes')

persisted_queries = {}

def generate_query_id(query):
    return hashlib.md5(query.encode('utf-8')).hexdigest()


def execute_query(query):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        return result
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        return []

class User(ObjectType):
    facultyID = String()
    name = String()
    department = String()
    country = String()
    city = String()
    address = String()
    appointment = String()
	
class Query(ObjectType):
    users = List(User)

    def resolve_users(self, info):
        query = "SELECT FACULTY_ID, Name, Department, COUNTRY, City, ADDRESS, APPOINTMENT FROM AdventureWorks2014.dbo.FACULTY"
        result = execute_query(query)
        return [User(
            facultyID=row[0],
            name=row[1],
            department=row[2],
            country=row[3],
            city=row[4],
            address=row[5],
            appointment=row[6]
        ) for row in result]

schema = Schema(query=Query)

schema = Schema(query=Query)

query = '''
query {
  users {
    facultyID
    name
    department
  }
}
'''

query_id = generate_query_id(query)
persisted_queries[query_id] = query

app = Flask(__name__)


@app.route("/persistedqueries", methods=["POST"])
def persisted_queries_endpoint():
    if request.method == 'POST':
        persisted_query_id = request.json.get('persistedQueryId')
        query = persisted_queries.get(persisted_query_id)

        if query:
            result = schema.execute(query)  
            return jsonify(result.data)
        else:
            return jsonify({'errors': [{'message': 'Persisted query not found.'}]})

    return jsonify({'message': 'Persisted queries endpoint'})



In [ ]:


curl -X POST \
  -H "Content-Type: application/json" \
  -d '{
    "persistedQueryId": "afbec3e19824099ac791ba23691143f5"
  }' \
  http://localhost:5000/persistedqueries
